# We also tried two neural network solutions, but neither worked properly

## This may be due to bad architecture or not enough training

In [3]:
import torch
from torch import nn
from torch.autograd import Variable
import pandas as pd
import dateutil
import datetime
from sklearn.model_selection import train_test_split
import numpy as np
import torch.nn.functional as funcs

In [4]:
# df = pd.read_csv('../data/Kiasma_sorted.csv', nrows=1000000, low_memory=False)
# df = pd.read_csv('../data/Rautatientoriitä_sorted.csv', low_memory=False)
df = pd.read_csv('../data/AaltoyliopistoMKorkeakouluaukio_sorted.csv', low_memory=False)

## Drop duplicated based on timestamp

In [5]:
df = df.drop_duplicates('ts').reset_index()

## Add some useful columns

In [6]:
# sort based on timestamp
df = df.sort_values('ts')
# the hour and minutes of the timestamp
df['hour_and_minutes'] = df['ts'].apply(lambda x: x[11:16])
# month of timestamp
df['month'] = df['ts'].apply(lambda x: x[5:7])
# the percentage of bikes
df['bikes_percent'] = df['bikes'] / df['total_slots']
df['minutes_from_midnight'] = df['ts'].apply(lambda x: int(x[11:13]) * 60 + int(x[14:16]))
df['hour'] = df['hour_and_minutes'].apply(lambda x: x[:2])
df['date'] = df['ts'].apply(pd.to_datetime)
df['weekday'] = df['date'].apply(lambda x: x.weekday())
df['weekend'] = df['weekday'].apply(lambda x: x > 4)
df.head()

,index,ts,lat,lon,operative,sid,name,bikes,total_slots,hour_and_minutes,month,bikes_percent,minutes_from_midnight,hour,date,weekday,weekend
0,0,2018-05-11T07:48:01,60.184312,24.826671,1,541,"Aalto-yliopisto (M), Korkeakouluaukio",5,40,07:48,05,0.125,468,07,2018-05-11 07:48:01,4,False
1,1,2018-05-11T07:49:01,60.184312,24.826671,1,541,"Aalto-yliopisto (M), Korkeakouluaukio",4,40,07:49,05,0.100,469,07,2018-05-11 07:49:01,4,False
2,2,2018-05-11T07:50:01,60.184312,24.826671,1,541,"Aalto-yliopisto (M), Korkeakouluaukio",4,40,07:50,05,0.100,470,07,2018-05-11 07:50:01,4,False
3,3,2018-05-11T07:51:01,60.184312,24.826671,1,541,"Aalto-yliopisto (M), Korkeakouluaukio",4,40,07:51,05,0.100,471,07,2018-05-11 07:51:01,4,False
4,4,2018-05-11T07:52:01,60.184312,24.826671,1,541,"Aalto-yliopisto (M), Korkeakouluaukio",4,40,07:52,05,0.100,472,07,2018-05-11 07:52:01,4,False


## Prepare for predictions

### Add column which states whether a bike was added at a given moment

In [7]:
df['bike_added'] = df.apply(lambda x: df.iloc[(x.name - 1 if x.name > 0 else 0)]['bikes'] < x['bikes'] , axis=1)
df['bike_removed'] = df.apply(lambda x: df.iloc[(x.name - 1 if x.name > 0 else 0)]['bikes'] > x['bikes'] , axis=1)
df.head(100)

,index,ts,lat,lon,operative,sid,name,bikes,total_slots,hour_and_minutes,month,bikes_percent,minutes_from_midnight,hour,date,weekday,weekend,bike_added,bike_removed
0,0,2018-05-11T07:48:01,60.184312,24.826671,1,541,"Aalto-yliopisto (M), Korkeakouluaukio",5,40,07:48,05,0.125,468,07,2018-05-11 07:48:01,4,False,False,False
1,1,2018-05-11T07:49:01,60.184312,24.826671,1,541,"Aalto-yliopisto (M), Korkeakouluaukio",4,40,07:49,05,0.100,469,07,2018-05-11 07:49:01,4,False,False,True
2,2,2018-05-11T07:50:01,60.184312,24.826671,1,541,"Aalto-yliopisto (M), Korkeakouluaukio",4,40,07:50,05,0.100,470,07,2018-05-11 07:50:01,4,False,False,False
3,3,2018-05-11T07:51:01,60.184312,24.826671,1,541,"Aalto-yliopisto (M), Korkeakouluaukio",4,40,07:51,05,0.100,471,07,2018-05-11 07:51:01,4,False,False,False
4,4,2018-05-11T07:52:01,60.184312,24.826671,1,541,"Aalto-yliopisto (M), Korkeakouluaukio",4,40,07:52,05,0.100,472,07,2018-05-11 07:52:01,4,False,False,False
5,5,2018-05-11T07:53:01,60.184312,24.826671,1,541,"Aalto-yliopisto (M), Korkeakouluaukio",4,40,07:53,05,0.100,473,07,2018-05-11 07:53:01,4,False,False,False
6,6,2018-05-11T07:54:01,60.184312,24.826671,1,541,"Aalto-yliopisto (M), Korkeakouluaukio",4,40,07:54,05,0.100,474,07,2018-05-11 07:54:01,4,False,False,False
7,7,2018-05-11T07:55:01,60.184312,24.826671,1,541,"Aalto-yliopisto (M), Korkeakouluaukio",4,40,07:55,05,0.100,475,07,2018-05-11 07:55:01,4,False,False,False
8,8,2018-05-11T07:56:01,60.184312,24.826671,1,541,"Aalto-yliopisto (M), Korkeakouluaukio",4,40,07:56,05,0.100,476,07,2018-05-11 07:56:01,4,False,False,False
9,9,2018-05-11T07:57:01,60.184312,24.826671,1,541,"Aalto-yliopisto (M), Korkeakouluaukio",4,40,07:57,05,0.100,477,07,2018-05-11 07:57:01,4,False,False,False


### Add max waiting time for next bike

In [8]:
# extract difference in minutes to a separate column
# on the moment that a bike was added, the waiting time is the time to next bike
i = 0
first_non_additive_index = 0
while (i < len(df)):
    while (i < len(df) and df.iloc[i]['bike_added'] == False):
        i += 1
    if (i >= len(df)):
        break
    additive_row = df.iloc[i]
    time = additive_row['minutes_from_midnight']
    difference = abs(time - df.iloc[first_non_additive_index]['minutes_from_midnight'])
    df.loc[first_non_additive_index:(i-1 if i > 0 else 0), 'max_wait_for_new_bike'] = difference 
    date_time = additive_row['date']
    last_date = df.iloc[first_non_additive_index]['date']
    date_difference = abs((date_time - last_date).total_seconds() / 60)
    df.loc[first_non_additive_index:(i-1 if i > 0 else 0), 'max_wait_for_new_bike_as_date'] = date_difference 
    first_non_additive_index = i
    i += 1
# if we can't know when the next bike will be added, fill with -1 for now
df = df.fillna(-1)
df.head(100)

KeyboardInterrupt: 

### Add idle time for each station

In [ ]:
i = 0
idle_for = 0
idle_for_as_date = 0
while (i < len(df)):
    row = df.iloc[i]
    previous_row = df.iloc[(i-1 if i > 0 else 0)]
    if (not row['bikes'] == previous_row['bikes']):
        idle_for = 0
        idle_for_as_date = 0
        df.loc[i, 'idle_for'] = idle_for
        df.loc[i, 'idle_for_as_date'] = idle_for_as_date        
        i+= 1
        continue
    time_now = row['minutes_from_midnight']
    time_last = previous_row['minutes_from_midnight']
    date_time_now = row['date']
    date_time_last = previous_row['date']
    idle_for += abs(time_now - time_last)
    secs = (date_time_now - date_time_last).total_seconds()
    idle_for_as_date += abs(secs / 60)
    df.loc[i, 'idle_for'] = idle_for
    df.loc[i, 'idle_for_as_date'] = idle_for_as_date
    i += 1
# if we can't know when the next bike will be taken, fill with -1 for now
df = df.fillna(-1)
df.head(50)

In [ ]:
df.describe()

### Add time for next bike take

Predict how much time for the next bike to be taken: max waiting time is not useful here.

In [ ]:
i = 0
first_non_removing_index = 0
while (i < len(df)):
    while (i < len(df) and df.iloc[i]['bike_removed'] == False):
        i += 1
    if (i >= len(df)):
        break
    removing_row = df.iloc[i]
    time = removing_row['minutes_from_midnight']
    times_until_now = df.loc[first_non_removing_index:i-1, 'minutes_from_midnight']
    difference = abs(time - times_until_now)
    df.loc[first_non_removing_index:(i-1 if i > 0 else 0), 'wait_for_bike_taken'] = difference 
    date_time = removing_row['date']
    date_times_until_now = df.loc[first_non_removing_index:i-1, 'date']
    date_difference = abs((date_time - date_times_until_now).apply(lambda x: x.total_seconds() / 60))
    df.loc[first_non_removing_index:(i-1 if i > 0 else 0), 'wait_for_bike_taken_as_date'] = date_difference 
    first_non_removing_index = i
    i += 1
# if we can't know when the next bike will be taken, fill with -1 for now
df = df.fillna(-1)
df.head(50)

## Remove useless or codependent columns

In [ ]:
df = df.drop(columns=['index', 'ts', 'sid', 'hour_and_minutes', 'name', 'bike_added', 'bike_removed', 'lat', 'lon', 'bikes', 'total_slots', 'date'])
print('using columns', df.columns.values)

## Define training and test sets

In [ ]:
# drop the columns we would not know in a real situation, as well as the column to be predicted
X = df.drop(columns=['max_wait_for_new_bike', 'wait_for_bike_taken']).as_matrix()
y = df['max_wait_for_new_bike'].as_matrix()
# set 
train_size = int(len(X)*0.8)

# define the training and test sets
X_train = torch.from_numpy(np.asarray(X[:train_size].astype('float')))
# take transpose to get right dimensions for torch
y_train = torch.from_numpy(np.asarray([y[:train_size]]).T)
X_test = torch.from_numpy(np.asarray(X[train_size:].astype('float')))
y_test = torch.from_numpy(np.asarray([y[train_size:]]).T)
# combine input and labels
train_data = torch.cat((X_train, y_train), 1)
test_data = torch.cat((X_test, y_test), 1)

# sanity check
assert len(X) == len(X_train) + len(X_test)

## Define a simple neural network

In [ ]:
class Net(nn.Module):
    def __init__(self, in_neurons=11, out_neurons=1, nr_layers=1, nr_neurons=10):
        super(Net, self).__init__()
        self.layers = nn.ModuleList()
        for i in range(nr_layers):
            self.layers.append(nn.Linear(in_neurons, nr_neurons))
            in_neurons = nr_neurons
        self.output_layer = nn.Linear(in_neurons, out_neurons)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
            x = funcs.relu(x)
        return self.output_layer(x)

## Define loss function

In [ ]:
loss_func = torch.nn.SmoothL1Loss()

## Train

In [ ]:
net = Net().double()
optimizer = torch.optim.SGD(net.parameters(), lr=0.05)
batch_size = 250
epochs = 500
for epoch in range(epochs):
    indices = torch.randperm(data_len)
    # train the data in batches
    for i in range(0, data_len, batch_size):
        batch_indices = indices[i: i+ batch_size]
        data = train_data[batch_indices]
        # assume that last cell in array is label
        inp, label = data[:, :data.shape[1]-1], data[:, data.shape[1]-1]
        # torch has to be fed variables
        input_var = Variable(inp)
        label_var = Variable(label.long())
        # reset parameters after each training round
        optimizer.zero_grad()
        # actual forward pass
        outputs = net(input_var)
        # calculate loss
        loss = loss_func(outputs, label_var)
        loss.backward()
        optimizer.step()
    if epoch % 50 == 0:
        print('e:', epoch)
        print('loss', loss.item())
        print('--')
print('training done with a training set of length', len(train_data))

## Evaluate with test set

In [ ]:
total_loss = 0
for i, test in enumerate(test_data):
    inp, label = test[:data.shape[1]-1], test[data.shape[1]-1]
    test_input_var = Variable(inp)
    test_label_var = Variable(label)
    test_outputs = net(test_input_var)
    # just print out the loss for now
    test_loss = loss_func(test_outputs, test_label_var)
    # this is basically just to see whether the net always predicted the same thing
    if i < 20:
        print('true label', test_label_var.item(), ', prediction', test_outputs.item())
    total_loss = test_loss.item()
print('average loss', total_loss/len(test_data))

## Use different loss

In [ ]:
loss_func = torch.nn.MSELoss()

In [ ]:
net = Net().double()
optimizer = torch.optim.SGD(net.parameters(), lr=0.05)
batch_size = 250
epochs = 500
for epoch in range(epochs):
    # train the data in batches
    batch_indices = torch.randperm(len(train_data))[:batch_size]
    data = train_data[batch_indices]
    # assume that last cell in array is label
    inp, label = data[:, :data.shape[1]-1], data[:, data.shape[1]-1]
    # torch has to be fed variables
    input_var = Variable(inp)
    label_var = Variable(label.reshape(label.shape[0], 1))
    # reset parameters after each training round
    optimizer.zero_grad()
    # actual forward pass
    outputs = net(input_var)
    # calculate loss
    loss = loss_func(outputs, label_var)
    if epoch % 50 == 0:
        print('e:', epoch)
        print('loss', loss.item())
        print('--')
    loss.backward()
    optimizer.step()
print('training done with a training set of length', len(train_data))

In [ ]:
total_loss = 0
for i, test in enumerate(test_data):
    inp, label = test[:data.shape[1]-1], test[data.shape[1]-1]
    test_input_var = Variable(inp)
    test_label_var = Variable(label)
    test_outputs = net(test_input_var)
    # just print out the loss for now
    test_loss = loss_func(test_outputs, test_label_var)
    if i < 20:
        print('true label', test_label_var.item(), ', prediction', test_outputs.item())
    total_loss = test_loss.item()
print('average loss', total_loss/len(test_data))

## The net wlays predicts the mean, so it's not capable of inferring anything valuable :(

# Treat as a classification problem: deifne ranges of waiting time

## Add classes for prediciton

In [63]:
def define_class(x):
    if x < 5:
        return 0
    if x < 15:
        return 1
    return 2

cleaned_df['max_wait_for_new_bike_class'] = cleaned_df['max_wait_for_new_bike'].apply(define_class)
# cleaned_df['wait_for_bike_taken_class'] = cleaned_df['wait_for_bike_taken_as_date'].apply(define_class)
# print('value count of classes in wait for next bike taken:')
# print(cleaned_df['wait_for_bike_taken_class'].value_counts())
print()
print('value count of classes in max wait for new bike')
print(cleaned_df['max_wait_for_new_bike_class'].value_counts())


value count of classes in max wait for new bike
0    34847
1     7154
2        1
Name: max_wait_for_new_bike_class, dtype: int64


## Define training and test sets

In [64]:
# drop the columns we would not know in a real situation, as well as the column to be predicted
X = cleaned_df.drop(columns=['max_wait_for_new_bike_class', 'max_wait_for_new_bike_as_date', 'wait_for_bike_taken_as_date']).as_matrix()
y = cleaned_df['max_wait_for_new_bike_class'].as_matrix()
# set 
train_size = int(len(X)*0.8)

# define the training and test sets
X_train = torch.from_numpy(np.asarray(X[:train_size].astype('float')))
# take transpose to get right dimensions for torch
y_train = torch.from_numpy(np.asarray([y[:train_size]]).astype('float').T)
X_test = torch.from_numpy(np.asarray(X[train_size:].astype('float')))
y_test = torch.from_numpy(np.asarray([y[train_size:]]).astype('float').T)
# combine input and labels
train_data = torch.cat((X_train, y_train), 1)
test_data = torch.cat((X_test, y_test), 1)

# sanity check
assert len(X) == len(X_train) + len(X_test)

## Define a simple neural network

In [65]:
class Net(nn.Module):
    def __init__(self, in_neurons=11, out_neurons=3, nr_layers=1, nr_neurons=10):
        super(Net, self).__init__()
        self.layers = nn.ModuleList()
        for i in range(nr_layers):
            self.layers.append(nn.Linear(in_neurons, nr_neurons))
            in_neurons = nr_neurons
        self.output_layer = nn.Linear(in_neurons, out_neurons)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
            x = funcs.relu(x)
        return self.output_layer(x)

## Define loss function

In [52]:
loss_func = torch.nn.CrossEntropyLoss()

## Train

In [66]:
net = Net().double()
optimizer = torch.optim.Adam(net.parameters(), lr=0.05)
data_len = len(train_data)
batch_size = 250
epochs = 50
for epoch in range(epochs):
    # shuffle the data
    indices = torch.randperm(data_len)
    # train the data in batches
    for i in range(0, data_len, batch_size):
        batch_indices = indices[i: i+ batch_size]
        data = train_data[batch_indices]
        # assume that last cell in array is label
        inp, label = data[:, :data.shape[1]-1], data[:, data.shape[1]-1]
        # torch has to be fed variables
        input_var = Variable(inp)
        label_var = Variable(label.long())
        # reset parameters after each training round
        optimizer.zero_grad()
        # actual forward pass
        outputs = net(input_var)
        # calculate loss
        loss = loss_func(outputs, label_var)
        loss.backward()
        optimizer.step()
    if epoch % 10 == 0:
        print('e:', epoch)
        print('loss', loss.item())
        print('--')

print('training done with a training set of length', len(train_data))

e: 0
loss 0.011217853767097214
--
e: 10
loss 6.320257158112769e-05
--
e: 20
loss 2.9847046717897215e-06
--
e: 30
loss 0.04352020912907271
--
e: 40
loss 0.006553049503505284
--
training done with a training set of length 33601


## Evaluate with test set

In [69]:
total_loss = 0
inp, label = test_data[:, :test_data.shape[1]-1], test_data[:, test_data.shape[1]-1]
test_input_var = Variable(inp)
test_label_var = Variable(label.long())
test_outputs = net(test_input_var)
# just print out the loss for now
test_loss = loss_func(test_outputs, test_label_var)
# this is basically just to see whether the net always predicted the same thing
total_loss = test_loss.item()
probs = funcs.softmax(test_outputs, dim=1)
predictions = torch.round(funcs.softmax(test_outputs, dim=1))
i = 0
for true, pred in zip(test_label_var.data, funcs.softmax(test_outputs, dim=1).data):
    if i == 200:
        break
    print(true, pred)
    i += 1

tensor(0) tensor([ 0.9742,  0.0257,  0.0001], dtype=torch.float64)
tensor(0) tensor([ 0.9742,  0.0257,  0.0001], dtype=torch.float64)
tensor(0) tensor([ 0.9742,  0.0257,  0.0001], dtype=torch.float64)
tensor(0) tensor([ 0.9742,  0.0257,  0.0001], dtype=torch.float64)
tensor(0) tensor([ 0.9742,  0.0257,  0.0001], dtype=torch.float64)
tensor(0) tensor([ 0.9742,  0.0257,  0.0001], dtype=torch.float64)
tensor(0) tensor([ 0.9742,  0.0257,  0.0001], dtype=torch.float64)
tensor(0) tensor([ 0.9742,  0.0257,  0.0001], dtype=torch.float64)
tensor(0) tensor([ 0.9742,  0.0257,  0.0001], dtype=torch.float64)
tensor(0) tensor([ 0.9742,  0.0257,  0.0001], dtype=torch.float64)
tensor(0) tensor([ 0.9742,  0.0257,  0.0001], dtype=torch.float64)
tensor(0) tensor([ 0.9742,  0.0257,  0.0001], dtype=torch.float64)
tensor(0) tensor([ 0.9742,  0.0257,  0.0001], dtype=torch.float64)
tensor(0) tensor([ 0.9742,  0.0257,  0.0001], dtype=torch.float64)
tensor(0) tensor([ 0.9742,  0.0257,  0.0001], dtype=torch.floa

## The net always predicts one label :(